# 最初を固定するSA

### パッケージ

In [1]:
ex_num = "003"

In [2]:
import os
# 環境によって変更
os.chdir('/home/jovyan/work/notebook')

# path = "../../output"
path = "../model"

In [3]:
import numpy as np
import pandas as pd
import os
import time
import csv

from tqdm.notebook import tqdm

import random
from typing import List, Callable
import itertools, math
from typing import List, Tuple

from utils import PerplexityCalculator

### データ読み込み

In [4]:
df = pd.read_csv("../input/sample_submission.csv")

In [5]:
sub_df = df.copy()
sub_df["best_value"] = 0

In [6]:
df

,id,text
0,0,advent chimney elf family fireplace gingerbrea...
1,1,advent chimney elf family fireplace gingerbrea...
2,2,yuletide decorations gifts cheer holiday carol...
3,3,yuletide decorations gifts cheer holiday carol...
4,4,hohoho candle poinsettia snowglobe peppermint ...
5,5,advent chimney elf family fireplace gingerbrea...


### 関数

In [7]:
def simulated_annealing(
    scorer,
    pattern_df,
    max_iterations: int = 1000,
    BATCH_SIZE: int = 32,
    initial_temperature: float = 100.0,
    cooling_rate: float = 0.99
) -> Tuple[List[str], float]:
    """
    焼きなまし法で最適な順序を探索する。

    Args:
        scorer: 評価関数を提供するオブジェクト。
        pattern_df: パターンのデータフレーム。
        max_iterations (int): 最大反復回数。
        BATCH_SIZE (int): バッチサイズ。
        initial_temperature (float): 初期温度。
        cooling_rate (float): 冷却率。

    Returns:
        solution_df: 最適な順序のデータフレーム。
    """
    
    solution_df = pattern_df.copy()
    solution_df["score"] = solution_df["score"].astype(float)
    
    words_len = len(solution_df.iloc[0]["text"].split())
    pattern_len = len(solution_df)
    
    temperature = initial_temperature
    
    perms = []

    for iteration in tqdm(range(max_iterations)):
        # 1行ずつ処理する
        for idx in range(pattern_len):
            current_sequence = solution_df.iloc[idx]["text"].split()
            
            neighbor_sequence = current_sequence[:]
            
            # 最初と最後の単語を固定し、ランダムに交換する
            i, j = random.sample(range(1, words_len), 2)
            neighbor_sequence[i], neighbor_sequence[j] = neighbor_sequence[j], neighbor_sequence[i]
            
            perms.append(" ".join(neighbor_sequence))
        
        # 評価値をまとめて計算
        neighbor_values = scorer.get_perplexity(perms, batch_size=min(pattern_len, BATCH_SIZE))

        # print(f"neighbor_values = {neighbor_values}")
        # 焼きなましによる評価値更新
        for idx in range(pattern_len):
            perm = perms[idx]
            neighbor_value = neighbor_values[idx]
            current_value = solution_df.iloc[idx]["score"]
            # print(f"current_value = {current_value}, neighbor_value = {neighbor_value}")
            
            # 確率的に次の状態を受け入れるかを決定
            delta = neighbor_value - current_value
            acceptance_probability = math.exp(-delta / temperature) if delta > 0 else 1.0

            # ベスト解を更新
            if (random.random() < acceptance_probability) or (neighbor_value < current_value):
                current_sequence = perm.split(" ")
                current_value = neighbor_value
                
                solution_df.loc[solution_df.index[idx], "text"] = perm
                solution_df.loc[solution_df.index[idx], "score"] = current_value
                start_word = solution_df.iloc[idx]["start"]
                # print(f"start_word = {start_word}, end_word = {end_word}, New best = {current_value} with '{current_sequence}'")

        # バッチリセット
        perms = []

        # 温度を更新
        if iteration % 10 == 0:
            temperature *= cooling_rate
                    
        # 温度が非常に低くなったら終了
        if temperature < 1e-5:
            break

        # iteration100回ごとに現在の温度を表示
        if iteration % 100 == 0:
            print(f"Iteration {iteration}, Temperature = {temperature}")
            
    return solution_df
     

In [8]:
def search_optimal_permutation(id: int, scorer, max_iterations: int = 1000, BATCH_SIZE=64):
    
    df = pd.read_csv("../input/sample_submission.csv")
    
    # 文字列を単語に分割
    words = df.loc[id,"text"].split()
    
    # データをタプルのリストとして準備
    pattern_data = []
    
    words_len = len(words)
    for i in range(0, words_len):
        new_words = words[:]
        if i != 0:
            new_words[i], new_words[0] = new_words[0], new_words[i]
        
        # new_words[0]意外の単語をシャッフル
        random.shuffle(new_words[1:])
        
        pattern_data.append((new_words[0], " ".join(new_words), 10000))
            
    pattern_df = pd.DataFrame(pattern_data, columns=["start", "text", "score"])        

    start = time.time()
    
    # SAで最適な順序を探索
    solutions_df = simulated_annealing(
        scorer,
        pattern_df,
        max_iterations=max_iterations,
        BATCH_SIZE=BATCH_SIZE,
        initial_temperature=100.0,
        cooling_rate=0.99
    )
    
    # optimized_dfのscore列の値が最小の行を取得
    best_sequence = solutions_df.loc[solutions_df["score"].idxmin(), "text"]
    best_value = solutions_df["score"].min()
    
    print(f"{id}th sample: {best_sequence}")
    print(f"Elapsed time: {time.time() - start:.2f} sec")
    print(f"Best value: {best_value}")
    
    return best_sequence, best_value, solutions_df


In [9]:
# 文字列型にのみダブルクォーテーションを付ける関数
def add_quotes_to_strings(value):
    if isinstance(value, str):  # 文字列型の場合
        return f'"{value}"'
    return value  # それ以外はそのまま

In [10]:
# LOAD GEMMA SCORER
scorer = PerplexityCalculator(f'{path}/gemma_2_9b')

cuda


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [11]:
# scorer.clear_gpu_memory()

In [12]:
for i in [3]:
    
    print(f"Processing {i}th sample")
    best_sequence, best_value, solutions_df = search_optimal_permutation(
        id=i,
        scorer=scorer,
        max_iterations=2**4,
        BATCH_SIZE=16
    )
    
    print(best_sequence)
    sub_df.loc[i, "best_value"] = best_value
    sub_df.loc[i, "text"] = best_sequence
    
    display(solutions_df)
    solutions_df.to_csv(f"{path}/out/solutions_{i}_{ex_num}.csv", index=False, header=True)



Processing 3th sample


  0%|          | 0/16 [00:00<?, ?it/s]

Iteration 0, Temperature = 99.0
3th sample: holiday decorations gifts cheer yuletide carol magi nutcracker polar grinch sleigh chimney ornament stocking holly workshop jingle beard naughty nice sing cheer and the of is unwrap visit relax eat
Elapsed time: 44.40 sec
Best value: 1100.9252592553103
holiday decorations gifts cheer yuletide carol magi nutcracker polar grinch sleigh chimney ornament stocking holly workshop jingle beard naughty nice sing cheer and the of is unwrap visit relax eat


/tmp/ipykernel_54/1435353653.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1100.9252592553103' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  sub_df.loc[i, "best_value"] = best_value


,start,text,score
0,yuletide,yuletide decorations holiday eat gifts carol m...,1380.869635
1,decorations,decorations yuletide gifts cheer holiday nutcr...,1127.032949
2,gifts,gifts decorations holiday cheer yuletide carol...,1343.623033
3,cheer,cheer decorations gifts yuletide sleigh carol ...,3051.649369
4,holiday,holiday decorations gifts cheer yuletide carol...,1100.925259
5,carol,carol decorations gifts relax holiday cheer ma...,2452.069155
6,magi,magi grinch polar cheer visit holly yuletide n...,1262.217028
7,nutcracker,nutcracker decorations is ornament holiday car...,1487.255122
8,polar,polar decorations gifts workshop holiday unwra...,2490.683625
9,grinch,grinch decorations and cheer holiday carol mag...,1223.382693


In [13]:
sub_df

,id,text,best_value
0,0,advent chimney elf family fireplace gingerbrea...,0.000000
1,1,advent chimney elf family fireplace gingerbrea...,0.000000
2,2,yuletide decorations gifts cheer holiday carol...,0.000000
3,3,holiday decorations gifts cheer yuletide carol...,1100.925259
4,4,hohoho candle poinsettia snowglobe peppermint ...,0.000000
5,5,advent chimney elf family fireplace gingerbrea...,0.000000


In [14]:
# 各セルに関数を適用
sub_df["text"] = sub_df["text"].astype(str)
sub_df = sub_df.applymap(add_quotes_to_strings)

/tmp/ipykernel_54/428122046.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  sub_df = sub_df.applymap(add_quotes_to_strings)


In [15]:
sub_df[["id", "text"]].to_csv(f"{path}/out/submission_{ex_num}.csv", index=False, header=True, quoting=csv.QUOTE_NONE)
sub_df.to_csv(f"{path}/out/score_{ex_num}.csv", index=False, header=True, quoting=csv.QUOTE_NONE)

In [16]:
np.mean(sub_df["best_value"])

np.float64(183.4875432092184)